This tutorial shows how to use the `NavData` class, including how to initialize
instances and perform basic operations.

In [ ]:
import gnss_lib_py as glp

In [ ]:
# Get data path of example file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/notebooks/tutorials/myreceiver.csv --quiet -O "myreceiver.csv"
data_path = "myreceiver.csv"

# NavData Operations

Create a NavData class from a csv file

In [ ]:
navdata = glp.NavData(csv_path=data_path)
print(navdata)

## Looping across a Time Row

You can use the `NavData.loop_time()` method to loop over groups of data that belong to same time stamp.

In [ ]:
for timestamp, delta_t, navdata_subset in glp.loop_time(navdata,'myTimestamp'):
    print('Current timestamp: ', timestamp)
    print('Difference between current and future time step', delta_t)
    print('Current group of data')
    print(navdata_subset)

## Concatenating NavData Instances

Use the `glp.concat()` method to concatenate two or more `NavData` instances. Each type of data is included in a row, so adding new rows with ``axis=0``, means adding new types of data. Concat requires that the new NavData matches the length of the existing NavData. Row concatenation assumes the same ordering within rows across both NavData instances (e.g. sorted by timestamp) and does not perform any matching/sorting itself. If the concatenating navdatas share a column name with ``axis=0`` then concat will add a suffix to create a unique row name.

In [ ]:
double_navdata = glp.concat(navdata, navdata, axis=0)
double_navdata

You can also concatenate new data to existing rows with ``axis=1``. If the row names of the new NavData instance don't match the row names ofthe existing NavData instance, the mismatched values will be filled with np.nan.

In [ ]:
glp.concat(double_navdata, navdata, axis=1)

## Sorting a NavData Instance based on Row Values

## Interpolate NaN values in a NavData Row